In [4]:
from dash import Dash, html, dcc, dash_table, callback_context
import dash_leaflet as dl
from dash.dependencies import Input, Output, State
from pyproj import CRS
from owslib.ogcapi.features import Features
import json
import pandas as pd
from osgeo import ogr, osr
from shapely.geometry import shape, Point

class MapLayers:
    @staticmethod
    def create_osm_layer():
        return dl.BaseLayer(
            dl.TileLayer(
                url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
                maxZoom=19,
                attribution="© OpenStreetMap"
            ),
            name="OpenStreetMap",
            checked=True
        )

    @staticmethod
    def create_esri_layer():
        return dl.BaseLayer(
            dl.TileLayer(
                url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
                maxZoom=19,
                attribution="© Esri"
            ),
            name="Esri Imagery",
            checked=False
        )

    @staticmethod
    def create_climate_stations_layer():
        return dl.Overlay(
            dl.WMSTileLayer(
                url="https://geo.weather.gc.ca/geomet-climate/?service=WMS&version=1.3.0&request=GetMap",
                layers="CLIMATE.STATIONS",
                format="image/png",
                transparent=True,
                attribution="Climate Stations",
                opacity=0.7
            ),
            name="Climate Stations",
            checked=True
        )

class MapCreator:
    @staticmethod
    def create_vancouver_map():
        return dl.Map(
            id="map-vancouver",
            style={'width': '100%', 'height': '50vh'},
            center=[49.2827, -123.1207],
            zoom=10,
            children=[
                dl.TileLayer(),
                dl.FeatureGroup([dl.EditControl(id='edit_control')]),
                dl.LayerGroup(id="markers-layer")
            ]
        )

    @staticmethod
    def create_calgary_map():
        return dl.Map(
            id="map-calgary",
            style={'width': '100%', 'height': '100vh'},
            center=[51.0447, -114.0719],
            zoom=10,
            children=[
                dl.TileLayer(
                    url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
                    maxZoom=19,
                    attribution="© OpenStreetMap"
                ),
                dl.WMSTileLayer(
                    url="https://geo.weather.gc.ca/geomet-climate/?service=WMS&version=1.3.0&request=GetMap",
                    layers="CLIMATE.STATIONS",
                    format="image/png",
                    transparent=True,
                    attribution="Climate Stations",
                    opacity=0.7
                )
            ]
        )

class Utilities:
    @staticmethod
    def get_epsg_code(lat, lon):
        crs_wgs84 = CRS("EPSG:4326")
        utm_zone = int((lon + 180) // 6) + 1
        hemisphere = 'north' if lat >= 0 else 'south'
        crs_utm = CRS.from_proj4(f"+proj=utm +zone={utm_zone} +datum=NAD83 +units=m +no_defs")
        return crs_utm.to_epsg()

    @staticmethod
    def create_markers(df):
        markers = []
        for _, row in df.iterrows():
            marker = dl.Marker(
                position=[row['LATITUDE_DD'], row['LONGITUDE_DD']],
                children=[
                    dl.Popup([
                        html.B(f"Station ID: {row['STN_ID']}"),
                        html.Br(),
                        html.B(f"Station Name: {row['STATION_NAME']}"),
                        html.Br(),
                        html.B(f"First Date: {row['FIRST_DATE']}"),
                        html.Br(),
                        html.B(f"Last Date: {row['LAST_DATE']}"),
                        html.Br(),
                        html.B(f"Elevation (m): {row['ELEVATION']}"),
                        html.Br(),
                        html.B(f"Has Hourly Data?: {row['HAS_HOURLY_DATA']}")
                    ])
                ]
            )
            markers.append(marker)
        return markers

    @staticmethod
    def points_within_polygon(points, polygon):
        poly = shape(polygon)
        return [point for point in points if poly.contains(Point(point))]

    @staticmethod
    def dms_to_dd(dms):
        sign = -1 if dms < 0 else 1
        coordinate_pos = abs(dms)
        degrees = coordinate_pos // 10000000
        minutes = (coordinate_pos // 100000) % 100
        seconds = (coordinate_pos % 100000) // 1000
        return sign * (degrees + (minutes / 60) + (seconds / 3600))

class DataProcessor:
    @staticmethod
    def process_station_data(bounds):
        south_west, north_east = bounds
        bbox = [south_west[1], south_west[0], north_east[1], north_east[0]]

        oafeat = Features("https://api.weather.gc.ca/")
        station_data = oafeat.collection_items("climate-stations", bbox=bbox)

        if "features" not in station_data:
            return None

        driver = ogr.GetDriverByName("GeoJSON")
        data_source = driver.Open(json.dumps(station_data), 0)
        layer = data_source.GetLayer()

        SRS_input = layer.GetSpatialRef()
        SR = osr.SpatialReference(str(SRS_input))
        epsg = SR.GetAuthorityCode(None)
        SRS_input.ImportFromEPSG(int(epsg))

        SRS_projected = osr.SpatialReference()
        SRS_projected.ImportFromEPSG(4326)

        transform = osr.CoordinateTransformation(SRS_input, SRS_projected)

        station_data_list = []
        for feature in layer:
            geom = feature.GetGeometryRef().Clone()
            geom.Transform(transform)
            field_names = [feature.GetFieldDefnRef(i).GetName() for i in range(feature.GetFieldCount())]
            field_values = [feature.GetField(name) for name in field_names]
            station_data_dict = dict(zip(field_names, field_values))
            station_data_list.append(station_data_dict)

        stations_df = pd.DataFrame(station_data_list)
        stations_df['LATITUDE_DD'] = stations_df['LATITUDE'].apply(Utilities.dms_to_dd)
        stations_df['LONGITUDE_DD'] = stations_df['LONGITUDE'].apply(Utilities.dms_to_dd)

        return stations_df

def create_app_layout():
    return html.Div([
        dcc.Tabs([
            dcc.Tab(label='Vancouver', children=[
                MapCreator.create_vancouver_map(),
                # html.Button("Reset", id="reset-button", n_clicks=0),  # Remove reset button
                html.Div(id='stations-table'),
                html.Div(id='filtered-data-storage', style={'display': 'none'}),
                html.Div(id='geojson-storage', style={'display': 'none'})
            ]),
            dcc.Tab(label='Calgary', children=[MapCreator.create_calgary_map()])
        ])
    ])

app = Dash(__name__)
app.layout = create_app_layout()

# Global variable to store the original full DataFrame
full_stations_df = None

@app.callback(
    [Output('map-vancouver', 'children'),
     Output('stations-table', 'children'),
     Output('filtered-data-storage', 'children'),
     Output('geojson-storage', 'children'),
     Output('edit_control', 'geojson')],
    [Input('map-vancouver', 'bounds'),
     Input('edit_control', 'geojson')],
    [State('filtered-data-storage', 'children'),
     State('geojson-storage', 'children')]
)
def update_map_and_table(bounds, geojson, stored_filtered_data, stored_geojson):
    global full_stations_df

    ctx = callback_context
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Reset only if no geojson (i.e., the user has deleted all drawings)
    if geojson is not None and not geojson['features']:
        stored_filtered_data = None
        stored_geojson = None
        geojson = {'type': 'FeatureCollection', 'features': []}

    if bounds is None:
        return [MapCreator.create_vancouver_map().children, None, stored_filtered_data, stored_geojson, geojson]

    # Fetch data only if bounds changes
    if trigger_id == 'map-vancouver':
        full_stations_df = DataProcessor.process_station_data(bounds)

    # Determine which DataFrame to use
    df_to_use = full_stations_df
    # Filter data if there's a polygon
    if geojson and geojson['features']:
        for feature in geojson['features']:
            if feature['geometry']['type'] in ['Polygon', 'Rectangle']:
                polygon = feature['geometry']
                markers_positions = list(zip(df_to_use['LONGITUDE_DD'], df_to_use['LATITUDE_DD']))
                inside_polygon_indices = [i for i, pos in enumerate(markers_positions) if Utilities.points_within_polygon([pos], polygon)]
                filtered_stations_df = df_to_use.iloc[inside_polygon_indices]
                df_to_use = filtered_stations_df
                stored_filtered_data = df_to_use.to_json(orient='split')
                stored_geojson = geojson

    # Create markers
    markers = Utilities.create_markers(df_to_use)

    # Create a Dash DataTable from the DataFrame
    table = dash_table.DataTable(
        columns=[
            {"name": 'Station ID', "id": 'STN_ID', "type": 'numeric'},
            {"name": 'Station Name', "id": 'STATION_NAME', "type": 'text'},
            {"name": 'First Date', "id": 'FIRST_DATE', "type": 'date'},
            {"name": 'Last Date', "id": 'LAST_DATE', "type": 'date'},
            {"name": 'Elevation (m)', "id": 'ELEVATION', "type": 'numeric'},
            {"name": 'Has Hourly Data?', "id": 'HAS_HOURLY_DATA', "type": 'text'},
        ],
        data=df_to_use.to_dict('records'),
        page_size=10
    )

    return [
        [
            dl.TileLayer(),
            dl.FeatureGroup([dl.EditControl(id="edit_control")]),  # EditControl is always enabled
            dl.LayerGroup(markers, id="markers-layer")
        ],
        table,
        stored_filtered_data,
        stored_geojson,
        geojson
    ]

if __name__ == '__main__':
    app.run_server(jupyter_mode='external', port=8050)

Dash app running on http://127.0.0.1:8050/
